# Part 4: Using GPU acceleration with PyTorch

In [2]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

    100% |████████████████████████████████| 92kB 3.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/6c/cb/69/466aef9cee879fb8f645bd602e34d45e754fb3dee2cb1a877a
Successfully built torchbearer


## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [3]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[-1.3541e+00, -3.7783e-01,  1.7075e-01,  ...,  8.6979e-01,
          2.1025e+00,  1.0457e-01],
        [ 3.4569e-03,  9.8455e-01,  3.3635e-02,  ...,  4.5593e-01,
         -4.8877e-01,  5.5726e-01],
        [ 5.4218e-01, -8.0904e-01,  3.1931e-01,  ...,  5.2237e-02,
         -9.5063e-01,  2.8600e+00],
        ...,
        [ 3.3639e-01, -2.9308e-01, -1.2841e+00,  ..., -1.0042e-01,
         -4.5187e-02, -2.0055e+00],
        [ 2.7589e-02,  1.5565e-01, -1.4837e+00,  ..., -1.6049e-01,
         -3.3928e-01, -1.1605e-01],
        [-7.9056e-02,  2.4880e+00,  2.0855e+00,  ..., -6.0211e-01,
         -3.1222e-01,  3.4849e-04]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [4]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[ 0.6542, -2.2201, -0.3584,  ..., -1.7723, -3.0471,  0.1022],
        [ 0.2601,  1.6342,  0.2866,  ..., -0.2465,  0.4182, -0.9709],
        [ 2.0489,  0.0921,  0.1105,  ...,  0.0532,  0.0731,  0.0731],
        ...,
        [ 2.2986,  0.1958, -0.0221,  ...,  1.3353, -1.3500, -0.0722],
        [ 0.2118,  0.6375, -0.0720,  ..., -0.3954,  1.4236, -0.4328],
        [ 2.0956,  0.7874,  0.1256,  ..., -1.2608,  0.6392,  0.5693]],
       device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[-1.1833e+00,  2.0334e-01, -2.7497e+00,  ...,  1.6804e-03,
         -2.8833e+00,  6.3089e-02],
        [ 9.9532e-01, -4.0814e-01, -1.7169e-01,  ...,  6.1525e-02,
          1.3659e-01,  6.7025e-02],
        [ 2.8730e-01,  1.7636e+00,  9.7379e-02,  ..., -7.9948e-02,
         -5.5221e-01, -9.0353e-02],
        ...,
        [-5.6354e-01, -1.6880e+00, -2.3608e-03,  ..., -7.9993e-01,
         -9.3477e-02,  1.1202e-01],
        [-1.4874e+00, -1.0647e+00, -1.6400e+00,  ...,  1.3190e+00,
          9.3260e-02,  3.3966e-02],
        [ 1.3083e-01,  6.5983e-03, -1.0108e+00,  ...,  3.2522e-01,
         -1.4569e+00,  1.3784e+00]], device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

GPU architecture is designed  to exploit computational concurrency. The ANN structure is composed uniformly by layers artificial neurons that perform the same computation. Because of this characteristics ANN models can be highly parallelized and perform better on GPUs instead of CPUs. 

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

The biggest limitation of GPUs is related to memory constraints. GPUs, in fact, have much smaller memory capacities compared to CPU. This can result in a barrier when working with large deep learning models. 